In [1]:
import numpy as np
from keras.models import load_model
from PIL import Image
import joblib
import cv2
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [2]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

In [3]:
cred = credentials.Certificate("ecohacks-522fa-firebase-adminsdk-uyybj-13c36d8578.json")
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [4]:
def add_to_firestore(prediction):
    doc_ref = db.collection('sample_accounts').document(u'OUR_ACCOUNT')
    doc = doc_ref.get()
    doc_dict = doc.to_dict()
    if prediction == 'Trash':
        doc_dict['trash_count'] += 1
        doc_dict['points'] += 1
    elif prediction == 'Recycle':
        doc_dict['recycle_count'] += 1
        doc_dict['points'] += 5
    elif prediction == 'Compost':
        doc_dict['compost_count'] += 1
        doc_dict['points'] += 3
    doc_ref.set(doc_dict)

In [5]:
model = load_model('model.h5')
le = joblib.load('labelEncoder.joblib')
font = cv2.FONT_HERSHEY_SIMPLEX

Metal device set to: Apple M1 Pro


2022-09-04 16:08:30.635537: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-04 16:08:30.635654: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
def predict(img):
    img = Image.fromarray(frame)
    img = img.resize((128, 128))
    img = np.array(img)
    img = img[:,:,::-1]
    plt.imshow(img)
    pred = model.predict(np.array([img]), verbose=0)
    pred_label = np.argmax(pred, axis=-1)
    pred_class = le.inverse_transform(pred_label)
    return pred_class[0], pred[0][pred_label][0]

In [7]:
def get_color(pred):
    if pred == 'Wall':
        return (175, 175, 175)
    elif pred == 'Trash':
        return (0, 0, 200)
    elif pred == 'Recycle':
        return (0, 200, 0)
    elif pred == 'Compost':
        return (0,233,255)

In [ ]:
vid = cv2.VideoCapture(2)

while(True):
    ret, frame = vid.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pred, confidence = predict(frame_rgb)
    color = get_color(pred)
    cv2.rectangle(frame, (100, 10), (1000, 130), (0, 0, 0), -1)
    cv2.putText(frame, f'{pred} ({round(confidence*100, 1)}%)', (100,100), font, 3, color, 2, cv2.LINE_AA)
    cv2.imshow('Detector', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    if cv2.waitKey(1) & 0xFF == ord('p'):
        print(f'Adding {pred} to firebase')
        add_to_firestore(pred)
        
vid.release()
cv2.destroyAllWindows()

2022-09-04 16:08:33.166891: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-04 16:08:33.214632: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Adding Trash to firebase
Adding Recycle to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Compost to firebase
Adding Recycle to firebase
Adding Wall to firebase
Adding Trash to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Compost to firebase
Adding Trash to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Recycle to firebase
Adding Trash to firebase
Adding Recycle to firebase
Adding Compost to firebase
